In [20]:
import pandas as pd
from surprise import Dataset, Reader, SVD, KNNBasic, accuracy
from surprise.model_selection import cross_validate, train_test_split, KFold
from collections import Counter, defaultdict
import numpy as np
import random

np.random.seed(10701)
random.seed(10701)

In [21]:
# Read Ratings Data
ratings_path = "ratings.dat"

ratings_df = pd.read_csv(
    ratings_path,
    sep="::",
    names=["UserID", "MovieID", "Rating", "Timestamp"],
    engine="python"
)

print(ratings_df.head())
print(ratings_df.shape)

   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291
(1000209, 4)


In [22]:
# Read Movies Data
movies_path = "movies.dat"

movies_df = pd.read_csv(
    movies_path,
    sep="::",
    names=["MovieID", "Title", "Genres"],
    engine="python",
    encoding="latin-1"
)

print(movies_df.head())
print(movies_df.shape)

   MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy
(3883, 3)


baseline models: 
 - userKNN, itemKNN: k = 20
 - MF: k = 20, epoches = 30, lr = 0.01, reg = 0.01
 - SVD: latent factors = 20, lr = 0.005, regularization = 0.02, epoches = 50

evaluation metrics:
 - MAE
 - RMSE
 - HR@10

In [23]:
def MAE(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

In [24]:
def eval(model, ratings_df, model_name, cv=5,random_state=10701, 
            cold_max_count=4, heavy_min_count=101, contr_min_std=1.5):
    
    reader = Reader(rating_scale=(1, 5))
    kf = KFold(n_splits=cv, random_state=random_state, shuffle=True)
    # item_summary = (ratings_df.groupby('MovieID')['Rating'].agg(count='size', std='std'))
    item_summary = ratings_df.groupby('MovieID').apply(
        lambda df: pd.Series({
            'count': len(df),
            'std': np.std(df['Rating'].to_numpy(dtype=float))
        })
    )

    cold_items = set(item_summary.index[item_summary['count'] <= cold_max_count])
    heavy_items = set(item_summary.index[item_summary['count'] >= heavy_min_count])
    contr_items = set(item_summary.index[item_summary['std'] > contr_min_std])

    results = {}

    def subset_eval(name, df):
        if df.empty:
            results[name] = None
            return
        data = Dataset.load_from_df(df[["UserID", "MovieID", "Rating"]], reader)
        cv_res = cross_validate(model, data, cv=kf, measures=['RMSE', 'MAE'], verbose=False)
        results[name] = {
            'mae': np.mean(cv_res['test_mae']),
            'rmse': np.mean(cv_res['test_rmse']),
            'count': len(df)
        }

    subset_eval('all', ratings_df)
    subset_eval('cold',  ratings_df[ratings_df['MovieID'].isin(cold_items)])
    subset_eval('heavy', ratings_df[ratings_df['MovieID'].isin(heavy_items)])
    subset_eval('controversial', ratings_df[ratings_df['MovieID'].isin(contr_items)])

    print(f"Results for {model_name}")

    for subset in ['all', 'cold', 'heavy', 'controversial']:
        print(f"\n{subset} Items:")
        res = results[subset]
        if res is None:
            print("No such items")
        else:
            print(f"  Count: {res['count']}")
            print(f"  MAE: {res['mae']:.4f}")
            print(f"  RMSE: {res['rmse']:.4f}")

    return results

In [25]:
# SVD

svd_model = SVD(
    biased=True,
    n_factors=20,
    n_epochs=50,
    lr_all=0.005,
    reg_all=0.02
)

svd_item_results = eval(svd_model, ratings_df, "SVD")


/var/folders/tr/grvbj7wn4nb6nxjf_k48bdk40000gn/T/ipykernel_4889/963771718.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  item_summary = ratings_df.groupby('MovieID').apply(


Results for SVD

all Items:
  Count: 1000209
  MAE: 0.6783
  RMSE: 0.8706

cold Items:
  Count: 598
  MAE: 0.9164
  RMSE: 1.1312

heavy Items:
  Count: 940925
  MAE: 0.6738
  RMSE: 0.8648

controversial Items:
  Count: 237
  MAE: 1.5119
  RMSE: 1.6738


In [26]:
# MF

mf_model = SVD(
    biased=False,
    n_factors=20,
    n_epochs=30,
    lr_all=0.01,
    reg_all=0.01
)

mf_item_results = eval(mf_model, ratings_df, "Matrix Factorization")

/var/folders/tr/grvbj7wn4nb6nxjf_k48bdk40000gn/T/ipykernel_4889/963771718.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  item_summary = ratings_df.groupby('MovieID').apply(


Results for Matrix Factorization

all Items:
  Count: 1000209
  MAE: 0.6925
  RMSE: 0.8917

cold Items:
  Count: 598
  MAE: 1.2255
  RMSE: 1.5602

heavy Items:
  Count: 940925
  MAE: 0.6876
  RMSE: 0.8858

controversial Items:
  Count: 237
  MAE: 1.5511
  RMSE: 1.8472


In [27]:
# UserKNN

userknn_model = KNNBasic(
    k=20, 
    sim_options={
        'name': 'pearson',
        'user_based': True
    }
)

userknn_item_results = eval(userknn_model, ratings_df, "UserKNN")


/var/folders/tr/grvbj7wn4nb6nxjf_k48bdk40000gn/T/ipykernel_4889/963771718.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  item_summary = ratings_df.groupby('MovieID').apply(


Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.

In [28]:
# ItemKNN

itemknn_model = KNNBasic(
    k=20, 
    sim_options={
        'name': 'pearson',
        'user_based': False
    }
)

itemknn_item_results = eval(itemknn_model, ratings_df, "ItemKNN")


/var/folders/tr/grvbj7wn4nb6nxjf_k48bdk40000gn/T/ipykernel_4889/963771718.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  item_summary = ratings_df.groupby('MovieID').apply(


Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.

In [29]:
def calculate_hr_at_k(model, ratings_df, model_name, k=10, test_size=0.2, random_state=10701):
    
    reader = Reader(rating_scale=(1, 5))
    dataset = Dataset.load_from_df(ratings_df[["UserID", "MovieID", "Rating"]], reader)
    trainset, testset = train_test_split(dataset, test_size=test_size, random_state=random_state)
    
    model.fit(trainset)

    user_testset = defaultdict(set)
    [user_testset[uid].add(iid) for uid, iid, _ in testset]
    anti_testset = trainset.build_anti_testset()
    preds = model.test(anti_testset)
    user_pred_scores = defaultdict(list)
    [user_pred_scores[p.uid].append((p.iid, p.est)) for p in preds]


    hits = 0
    num_evaluated = 0

    for uid in sorted(user_testset):

        preds = user_pred_scores.get(uid)
        if preds is None:
            continue
            
        num_evaluated += 1
        ground_truth_items = user_testset[uid]
        top_k = [iid for iid, _ in sorted(preds, key=lambda x: (-x[1], x[0]))[:k]]

        if ground_truth_items.intersection(top_k):
            hits += 1

    hr = hits / num_evaluated if num_evaluated > 0 else 0.0

    print(f"{model_name}: {hr:.4f}")
    return hr


print("HR@10 on All Data")

hr10_results = {}

models = {
    "SVD": svd_model,
    "MF": mf_model,
}

for model_name, model in models.items():
    hr10_results[model_name] = calculate_hr_at_k(model, ratings_df, model_name, k=10)



HR@10 on All Data
SVD: 0.4127
MF: 0.3943
